In [37]:
import pandas as pd
# 승하차 인원 파악
data = pd.read_csv(r'C:\\sqlite\\mysql\\code\\seoul\\seoul-metro-2021\\seoul-metro-2021.logs.csv')
data

,timestamp,station_code,people_in,people_out
0,2021-01-01T05:00:00.000+09:00,150,86,85
1,2021-01-01T06:00:00.000+09:00,150,111,355
2,2021-01-01T07:00:00.000+09:00,150,157,438
3,2021-01-01T08:00:00.000+09:00,150,306,592
4,2021-01-01T09:00:00.000+09:00,150,333,841
...,...,...,...,...
1941548,2021-12-31T19:00:00.000+09:00,2828,74,263
1941549,2021-12-31T20:00:00.000+09:00,2828,73,145
1941550,2021-12-31T21:00:00.000+09:00,2828,95,209
1941551,2021-12-31T22:00:00.000+09:00,2828,54,138


In [38]:
# 지하철역정보 파악
station_info = pd.read_csv(r'C:\\sqlite\\mysql\\code\\seoul\\seoul-metro-2021\\seoul-metro-station-info.csv')


In [39]:
data[['people_in','people_out']]

,people_in,people_out
0,86,85
1,111,355
2,157,438
3,306,592
4,333,841
...,...,...
1941548,74,263
1941549,73,145
1941550,95,209
1941551,54,138


In [40]:
station_sum = data.groupby(data['station_code']).sum()

In [41]:
station_info = station_info[['station.code','geo.latitude','geo.longitude','station.name']]

In [42]:
station_info = station_info.set_index('station.code')

In [43]:
joined_data = station_sum.join(station_info)

In [44]:
morning_data11 = data[pd.to_datetime(data['timestamp']).dt.hour >= 11]

In [45]:
morning_data2 = morning_data11[pd.to_datetime(data['timestamp']).dt.hour <= 14]

C:\Users\admin\AppData\Local\Temp\ipykernel_11416\1863904951.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  morning_data2 = morning_data11[pd.to_datetime(data['timestamp']).dt.hour <= 14]


In [46]:
morning_station_sum = morning_data2.groupby("station_code").sum()

In [47]:
morning_joined_data = morning_station_sum.join(station_info)

In [48]:
import folium
from folium.plugins import HeatMap
morning_seoul_in = folium.Map(location=[37.55, 126.98], zoom_start = 12)
HeatMap(data = morning_joined_data[['geo.latitude', 'geo.longitude', 'people_in']]).add_to(morning_seoul_in)
morning_seoul_in

In [ ]:
morning_seoul_in = folium.Map(location=[37.55, 126.98], zoom_start = 12)
HeatMap(data = morning_joined_data[['geo.latitude', 'geo.longitude', 'people_out']]).add_to(morning_seoul_in)
morning_seoul_in

In [ ]:
lunch_seoul_out_circle = folium.Map(location=[37.55, 126.98], zoom_start = 12)

for index, row in morning_joined_data[['geo.latitude', 'geo.longitude', 'people_out']].iterrows():
    if int(row['people_out']) > 2_500_000:
        folium.Circle(location=[str(row['geo.latitude']), str(row['geo.longitude'])],   color='Red', fill=True,
              radius=(int(row['people_out'])/10000)).add_to(lunch_seoul_out_circle)

lunch_seoul_out_circle

In [61]:
tiles = "http://mt0.google.com/vt/lyrs=h&hl=ko&x={x}&y={y}&z={z}"
attr = "Google"
lunch_seoul_out_circle = folium.Map(location=[37.55, 126.98], zoom_start = 12, tiles = tiles, attr = attr)

for index, row in morning_joined_data[['geo.latitude', 'geo.longitude', 'people_out', 'station.name']].iterrows():
    if int(row['people_out']) > 2_500_000:
        folium.Circle(location=[str(row['geo.latitude']), str(row['geo.longitude'])],   color='Red', fill=True,
              radius=(int(row['people_out'])/10000)).add_to(lunch_seoul_out_circle)
        folium.Marker(location=[str(row['geo.latitude']), str(row['geo.longitude'])], tooltip=[str(row['station.name'])], icon=folium.Icon('red')).add_to(lunch_seoul_out_circle)

lunch_seoul_out_circle